## PF

In [ ]:
#=================== Imports ===================
import pickle
import open3d as o3d
import numpy as np
import math
import random
import json
import yaml
import uuid
from scipy.optimize import linear_sum_assignment

#=================== Particle Filter Tracker ===================
class ParticleBoxTracker:
    def __init__(self, bbox_3d, num_particles=1000000):
        bbox_3d = np.array(bbox_3d)
        self.num_particles = num_particles

        # Create particle array: [x, y, z, vx, vy, vz]
        self.particles = np.zeros((num_particles, 6))
        self.particles[:, 0:3] = bbox_3d[:3] + np.random.normal(0, 0.9, (num_particles, 3))
        self.particles[:, 3:6] = 0  # Start with zero velocity
        self.weights = np.ones(num_particles) / num_particles

        # State transition model (with time step dt)
        self.dt = 1.0
        self.F = np.array([[1, 0, 0, self.dt, 0, 0],
                           [0, 1, 0, 0, self.dt, 0],
                           [0, 0, 1, 0, 0, self.dt],
                           [0, 0, 0, 1, 0, 0],
                           [0, 0, 0, 0, 1, 0],
                           [0, 0, 0, 0, 0, 1]])

        self.process_noise_std = np.array([0.9, 0.9, 0.9, 0.9, 0.9, 0.9])
        self.measurement_noise_std = 0.9

    def predict(self):
        # Move particles forward using state model and noise
        noise = np.random.normal(0, self.process_noise_std, self.particles.shape)
        self.particles = self.particles @ self.F.T + noise

    def update(self, bbox_3d):
        # Update weights based on how close each particle is to the new measurement
        measurement = np.array(bbox_3d[:3])
        distances = np.linalg.norm(self.particles[:, :3] - measurement, axis=1)
        self.weights *= (1.0 / (np.sqrt(2 * np.pi) * self.measurement_noise_std)) * \
                        np.exp(-0.5 * (distances / self.measurement_noise_std) ** 2)
        self.weights += 1e-300  # Avoid zeros
        self.weights /= np.sum(self.weights)

        # Resample if effective sample size is too low
        effective_n = 1.0 / np.sum(self.weights ** 2)
        if effective_n < self.num_particles / 2:
            self.resample()

    def resample(self):
        # Standard resampling from weighted particles
        cumulative_sum = np.cumsum(self.weights)
        cumulative_sum[-1] = 1.0
        indexes = np.searchsorted(cumulative_sum, np.random.rand(self.num_particles))
        self.particles = self.particles[indexes]
        self.weights.fill(1.0 / self.num_particles)

    def get_state(self):
        # Return the weighted average state
        state = np.average(self.particles, weights=self.weights, axis=0)
        return state[:3]

#=================== Main Tracker Logic ===================
class bb_traking():
    def __init__(self, config) -> None:
        self.lidar_frequency = config.get('lidar_frequency', 10)
        self.countdown = config.get('countdown', 1)
        self.velocity_threshold = config.get('velocity_threshold', 1.0)
        self.ignore_velocity = config.get('ignore_velocity', False)
        self.box_colors = config.get('box_colors', [[1,0,0],[0,1,0],[0,0,1]])
        self.enable_visualization = config.get('enable_visualization', False)
        self.x_threshold = config.get('x_threshold', 20)
        self.y_threshold = config.get('y_threshold', 3.5)
        self.lost_association_threshold = config.get('lost_association_threshold', 5.0)
        self.lost_frame_threshold = config.get('lost_frame_threshold', 5)

        self.vehicle_db = {}     # Active tracks
        self.lost_tracks = {}    # Temporarily lost tracks

    def create_rotation_matrix(self, yaw):
        cos_yaw = np.cos(yaw)
        sin_yaw = np.sin(yaw)
        return np.array([
            [cos_yaw, -sin_yaw, 0],
            [sin_yaw,  cos_yaw, 0],
            [0,        0,       1]
        ])

    def calculate_velocity(self, P1, P2):
        t = 1 / self.lidar_frequency
        return (np.array(P2[:2]) - np.array(P1[:2])) / t

    def generate_tracking_uuid(self, box, velocity):
        new_uuid = str(uuid.uuid4())
        tracker = ParticleBoxTracker(box)
        tracker.predict()
        predicted_bbox_center = tracker.get_state().tolist()
        self.vehicle_db[new_uuid] = [
            *predicted_bbox_center,
            0, float('inf'), box, tracker,
            self.countdown, velocity, True
        ]
        return new_uuid

    def create_bb_box(self, box, uuid_key, point_cloud=None):
        center = np.array(box[:3])
        extent = np.array(box[3:6])
        yaw = box[6]
        rotation = self.create_rotation_matrix(yaw)
        bbox_3d = o3d.geometry.OrientedBoundingBox(center, rotation, extent)
        color_idx = abs(hash(uuid_key)) % len(self.box_colors)
        color = self.box_colors[color_idx]
        bbox_3d.color = color
        res_box = {
            'uuid': uuid_key,
            'label': 'Car',
            'position': {'x': float(center[0]), 'y': float(center[1]), 'z': float(center[2])},
            'dimensions': {'x': float(extent[1]), 'y': float(extent[0]), 'z': float(extent[2])},
            'yaw': float(yaw),
            'stationary': True,
            'camera_used': None,
            'attributes': {'state': 'Parked'},
            'points_count': 0,
            'key': uuid_key,
            'color': color
        }
        if point_cloud is not None:
            res_box['points_count'] = self.count_points_in_bbox(point_cloud, bbox_3d)
        return bbox_3d, res_box

    def count_points_in_bbox(self, point_cloud, bbox):
        cropped_pc = point_cloud.crop(bbox)
        return len(np.asarray(cropped_pc.points))

    #=================== Main Tracking per Frame ===================
    def visualize_bounding_boxes(self, frames, point_clouds=None):
        o3d_boxes = []
        frame_results = []

        for f, frame_boxes in enumerate(frames):
            current_point_cloud = point_clouds[f] if point_clouds is not None else None
            for key in list(self.vehicle_db.keys()):
                self.vehicle_db[key][3] = 0  # Reset mapped_count

            old_keys = list(self.vehicle_db.keys())
            new_boxes = list(frame_boxes)
            M = len(new_boxes)
            box_results = []
            unassigned_indices = set(range(M))

            #=== Match new boxes to existing tracks ===#
            for track_id in old_keys:
                track_data = self.vehicle_db[track_id]
                old_center = track_data[:3]
                best_distance = float('inf')
                best_idx = None

                for idx in unassigned_indices:
                    box = new_boxes[idx]
                    center = box[:3]
                    dx = abs(center[0] - old_center[0])
                    dy = abs(center[1] - old_center[1])
                    if dx <= self.x_threshold and dy <= self.y_threshold:
                        distance = math.sqrt(dx*dx + dy*dy)
                        if distance < best_distance:
                            best_distance = distance
                            best_idx = idx

                if best_idx is not None:
                    unassigned_indices.remove(best_idx)
                    box = new_boxes[best_idx]
                    tracker = track_data[6]
                    tracker.update(box)
                    tracker.predict()
                    predicted_bbox_center = tracker.get_state().tolist()
                    old_center = track_data[:3]
                    self.vehicle_db[track_id][:3] = predicted_bbox_center
                    self.vehicle_db[track_id][3] += 1
                    self.vehicle_db[track_id][4] = best_distance
                    self.vehicle_db[track_id][5] = box
                    if not self.ignore_velocity:
                        new_velocity = self.calculate_velocity(old_center, box[:3])
                        self.vehicle_db[track_id][8] = new_velocity
                    bb, bb_res = self.create_bb_box(box, track_id, point_cloud=current_point_cloud)
                    o3d_boxes.append(bb)
                    box_results.append(bb_res)
                else:
                    self.vehicle_db[track_id][7] -= 1
                    if self.vehicle_db[track_id][7] <= -1:
                        self.lost_tracks[track_id] = {'data': self.vehicle_db[track_id], 'lost_count': 0}
                        del self.vehicle_db[track_id]

            #=== Try to reconnect with lost tracks ===#
            if self.lost_tracks:
                for idx in list(unassigned_indices):
                    box = new_boxes[idx]
                    best_lost_key = None
                    best_distance = float('inf')
                    for lost_key in list(self.lost_tracks.keys()):
                        lost_track = self.lost_tracks[lost_key]['data']
                        lost_center = lost_track[:3]
                        dx = abs(box[0] - lost_center[0])
                        dy = abs(box[1] - lost_center[1])
                        if dx <= self.x_threshold and dy <= self.y_threshold:
                            distance = math.sqrt(dx*dx + dy*dy)
                            if distance < best_distance:
                                best_distance = distance
                                best_lost_key = lost_key
                    if best_lost_key and best_distance < self.lost_association_threshold:
                        tracker = self.lost_tracks[best_lost_key]['data'][6]
                        tracker.update(box)
                        tracker.predict()
                        predicted_bbox_center = tracker.get_state().tolist()
                        old_center = self.lost_tracks[best_lost_key]['data'][:3]
                        self.lost_tracks[best_lost_key]['data'][:3] = predicted_bbox_center
                        self.lost_tracks[best_lost_key]['data'][3] += 1
                        self.lost_tracks[best_lost_key]['data'][4] = best_distance
                        self.lost_tracks[best_lost_key]['data'][5] = box
                        if not self.ignore_velocity:
                            new_velocity = self.calculate_velocity(old_center, box[:3])
                            self.lost_tracks[best_lost_key]['data'][8] = new_velocity
                        self.vehicle_db[best_lost_key] = self.lost_tracks[best_lost_key]['data']
                        del self.lost_tracks[best_lost_key]
                        bb, bb_res = self.create_bb_box(box, best_lost_key, point_cloud=current_point_cloud)
                        o3d_boxes.append(bb)
                        box_results.append(bb_res)
                        unassigned_indices.remove(idx)

            #=== Start new tracks for unmatched boxes ===#
            for idx in unassigned_indices:
                box = new_boxes[idx]
                new_key = self.generate_tracking_uuid(box, [0, 0])
                bb, bb_res = self.create_bb_box(box, new_key, point_cloud=current_point_cloud)
                o3d_boxes.append(bb)
                box_results.append(bb_res)

            #=== Clean up long-lost tracks ===#
            lost_keys_to_delete = []
            for key in self.lost_tracks:
                self.lost_tracks[key]['lost_count'] += 1
                if self.lost_tracks[key]['lost_count'] > self.lost_frame_threshold:
                    lost_keys_to_delete.append(key)
            for key in lost_keys_to_delete:
                del self.lost_tracks[key]

            frame_results.append({'cuboids': box_results})

        if self.enable_visualization:
            o3d.visualization.draw_geometries(o3d_boxes)

        return frame_results

#=================== Helpers ===================
def load_bounding_boxes(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

def convert_to_python_types(data):
    if isinstance(data, dict):
        return {k: convert_to_python_types(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_to_python_types(i) for i in data]
    elif isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, (np.float32, np.float64)):
        return float(data)
    elif isinstance(data, (np.int32, np.int64)):
        return int(data)
    return data

#=================== Run Tracking ===================
if __name__ == "__main__":
    box_list = load_bounding_boxes("box_list(kopia).pkl")

    config = {
       'x_threshold': 20,
       'y_threshold': 3.5,
       'lidar_frequency': 10,
       'enable_visualization': False,
       'lost_association_threshold': 5.0,
       'lost_frame_threshold': 5
    }

    tracker = bb_traking(config)
    frame_results = tracker.visualize_bounding_boxes(box_list, point_clouds=None)

    converted_results = convert_to_python_types(frame_results)
    output_path = "PF_1M_final.json"
    with open(output_path, 'w') as json_file:
        json.dump(converted_results, json_file, indent=4)

    print(f"Done! Results saved in {output_path}")
